In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pickle
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

In [ ]:
#Reading Files


X_test_filename = '/content/drive/MyDrive/IE434 Deep Dive 11 project/Deep Dive 4 (Milestone 3)/X_test.pkl'

with open(X_test_filename, 'rb') as file:
    X_test = pickle.load(file)


y_test_filename = '/content/drive/MyDrive/IE434 Deep Dive 11 project/Deep Dive 4 (Milestone 3)/y_test.pkl'

with open(y_test_filename, 'rb') as file:
    y_test = pickle.load(file)

y_test_df = '/content/drive/MyDrive/IE434 Deep Dive 11 project/Deep Dive 4 (Milestone 3)/y_test_numerical.pkl'

with open(y_test_df, 'rb') as file:
    y_test_df = pickle.load(file)

## Architecture of Final model


In [ ]:
class ImprovedGRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, dropout_rate=0.2):
        super(ImprovedGRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # GRU Layer
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=(dropout_rate if num_layers > 1 else 0))

        # Dropout layer
        self.dropout = nn.Dropout(dropout_rate)

        # Fully connected Layer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        out, hidden = self.gru(x, hidden)

        # Applying dropout to the output of the last GRU layer
        out = self.dropout(out)

        # Reshape output for the fully connected layer
        out = out.reshape(-1, self.hidden_size)

        # Get the final output
        out = self.fc(out)

        return out, hidden

    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        return hidden


## Calculation of Confusion Matrix for Trained Model

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import confusion_matrix

# Convert X_test DataFrame to PyTorch tensor
X_test_tensor = torch.tensor(X_test.values).float()
y_test_tensor = torch.tensor(y_test_df.values).long()  # Directly use y_test if it's already encoded

# DataLoader for test set
batch_size = 64  # Adjust as needed
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=batch_size, shuffle=False)

# Load the model
model_GRU = torch.load('/content/drive/MyDrive/IE434 Deep Dive 11 project/Deep Dive 4 (Milestone 3)/model_GRU.pth')
model_GRU.eval()

# Initialize variables to store predictions and true labels
all_predictions = []
all_true_labels = []

# Make predictions
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        # Reshape X_batch to [batch_size, seq_len, features]
        X_batch = X_batch.unsqueeze(1)  # Reshape to [64, 1, 23]

        batch_size = X_batch.size(0)
        hidden = model_GRU.init_hidden(batch_size)

        output, hidden = model_GRU(X_batch, hidden)
        predictions = output.argmax(dim=1)
        all_predictions.extend(predictions.cpu().numpy())
        all_true_labels.extend(y_batch.cpu().numpy())

# Check lengths of all_predictions and all_true_labels
print(f"Length of all_predictions: {len(all_predictions)}")
print(f"Length of all_true_labels: {len(all_true_labels)}")

print('\n')
print('Confusion Matrix:')
# Compute confusion matrix
conf_matrix = confusion_matrix(all_true_labels, all_predictions)
print(conf_matrix)


Length of all_predictions: 9144
Length of all_true_labels: 9144
[[1393    3  857]
 [  44 1392  691]
 [ 271  349 4144]]


In [ ]:
#Accuracy

import numpy as np

accuracy = np.trace(conf_matrix) / np.sum(conf_matrix)
print("Accuracy:", accuracy)

Accuracy: 0.7577646544181977
